# Build ChatBot

From: [Build ChatBot Tutorial](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)

In [5]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage


In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
resp1 = model.invoke([HumanMessage(content="Hi! I'm Bob")])
resp1.content

'Hello Bob! How can I assist you today?'

In [6]:
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. How can I assist you today, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 35, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c115acb1-9703-4b61-8dcc-291b330f0de0-0')

## Message History

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable": {"session_id": "abc2"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello Bob! How can I assist you today?'

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob, as you mentioned earlier. How can I help you, Bob?'

## Prompt Template

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [14]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})
response.content

'Hello Bob! How can I assist you today?'

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [16]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola, Bob! ¿En qué puedo ayudarte hoy?'